<a href="https://colab.research.google.com/github/farhantanvir1/Anti-COVID/blob/main/costco_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports

In [2]:
import os
import json
import urllib.request
import zipfile
from pprint import pprint

import numpy as np
import tensorflow as tf
import keras as k

In [3]:
pip install tensorflow

# Load data

In [5]:
import urllib.request

data_folder = 'data'

if not os.path.exists(data_folder):
  url = 'https://github.com/USC-Melady/KDD19-CoSTCo/releases/download/demo_data/data.zip'
  filedata = urllib2.urlopen(url)

  with open('data.zip', 'wb') as f:
      f.write(filedata.read())

  with zipfile.ZipFile('data.zip', 'r') as f:
    f.extractall('.')

NameError: name 'urllib2' is not defined

In [ ]:
shape = np.loadtxt(os.path.join(data_folder, 'tensor_shape.txt')).astype(int)
tr_idxs = np.loadtxt(os.path.join(data_folder, 'train_indices.txt')).astype(int)
tr_vals = np.loadtxt(os.path.join(data_folder, 'train_values.txt'))
te_idxs = np.loadtxt(os.path.join(data_folder, 'test_indices.txt')).astype(int)
te_vals = np.loadtxt(os.path.join(data_folder, 'test_values.txt'))

# Define util functions

In [ ]:
def mape_keras(y_true, y_pred, threshold=0.1):
    v = k.backend.clip(k.backend.abs(y_true), threshold, None)
    diff = k.backend.abs((y_true - y_pred) / v)
    return 100.0 * k.backend.mean(diff, axis=-1)

def mae(y_true, y_pred):
    return np.mean(np.abs(y_pred - y_true))

def rmse(y_true, y_pred):
    return np.sqrt(np.mean(np.square(y_pred - y_true)))

def mape(y_true, y_pred, threshold=0.1):
    v = np.clip(np.abs(y_true), threshold, None)
    diff = np.abs((y_true - y_pred) / v)
    return 100.0 * np.mean(diff, axis=-1).mean()

def transform(idxs):
    return [idxs[:, i] for i in range(idxs.shape[1])]

def set_session(device_count=None, seed=0):
    gpu_options = tf.GPUOptions(allow_growth=True)
    if device_count is not None:
        config = tf.ConfigProto(
            gpu_options=gpu_options,
            device_count=device_count
        )
    else:
        config = tf.ConfigProto(gpu_options=gpu_options)
    sess = tf.Session(config=config)
    k.backend.set_session(sess)

    np.random.seed(seed)
    tf.set_random_seed(seed)
    return sess

def get_metrics(model, x, y, batch_size=1024):
    yp = model.predict(x, batch_size=batch_size, verbose=1).flatten()
    return {
        "rmse": float(rmse(y, yp)),
        "mape": float(mape(y, yp)),
        "mae": float(mae(y, yp))
    }

# Create a CoSTCo model

In [ ]:
def create_costco(shape, rank, nc):
    inputs = [k.Input(shape=(1,), dtype="int32") for i in range(len(shape))]
    embeds = [
        k.layers.Embedding(output_dim=rank, input_dim=shape[i])(inputs[i])
        for i in range(len(shape))
    ]
    x = k.layers.Concatenate(axis=1)(embeds)
    x = k.layers.Reshape(target_shape=(rank, len(shape), 1))(x)
    x = k.layers.Conv2D(
        nc,
        kernel_size=(1, len(shape)),
        activation="relu",
        padding="valid"
    )(x)
    x = k.layers.Conv2D(
        nc,
        kernel_size=(rank, 1),
        activation="relu",
        padding="valid"
    )(x)
    x = k.layers.Flatten()(x)
    x = k.layers.Dense(nc, activation="relu")(x)
    outputs = k.layers.Dense(1, activation="relu")(x)
    model = k.Model(inputs=inputs, outputs=outputs)

    return model

# Set hyper-parameters

In [ ]:
lr = 1e-4
rank = 20
nc = rank
epochs = 50
batch_size = 256

seed = 3
verbose = 1

# Train with early stopping

In [ ]:
set_session(device_count={"GPU": 0}, seed=seed)
optim = k.optimizers.Adam(lr=lr)

model = create_costco(shape, rank, nc)
model.compile(optim, loss=["mse"], metrics=["mae", mape_keras])
hists = model.fit(
    x=transform(tr_idxs),
    y=tr_vals,
    verbose=verbose,
    epochs=epochs,
    batch_size=batch_size,
    validation_split=0.1,
    callbacks=[k.callbacks.EarlyStopping(
        monitor="val_mean_absolute_error",
        patience=10,
        restore_best_weights=True)],
);

# Evaluation model prediction over the test set

In [ ]:
tr_info = get_metrics(model, transform(tr_idxs), tr_vals)
te_info = get_metrics(model, transform(te_idxs), te_vals)

In [ ]:
pprint({'train': tr_info, 'test': te_info})

In [1]:
import os
import numpy as np
import tensorflow as tf
from tensorflow import keras as k
from sklearn.metrics import f1_score, precision_score, recall_score, roc_auc_score, average_precision_score

# Load DMD dataset
data_file = "/content/adj_del_4mic_myid.txt"

# Load triplet associations (drug, microbe, disease, label)
data = np.loadtxt(data_file).astype(int)

# Extract triplet indices and labels
d_indices = data[:, 0]  # Drug indices
m_indices = data[:, 1]  # Microbe indices
n_indices = data[:, 2]  # Disease indices
labels = data[:, 3].astype(float)  # Convert to float for label smoothing

# Get dataset shape (number of unique drugs, microbes, diseases)
num_drugs = np.max(d_indices) + 1
num_microbes = np.max(m_indices) + 1
num_diseases = np.max(n_indices) + 1
shape = [num_drugs, num_microbes, num_diseases]

# Convert dataset into a set for fast lookup
positive_set = set(tuple(row[:3]) for row in data)

# Generate negative samples
num_neg_samples = len(data)  # 1:1 ratio of negative to positive
negative_samples = []
while len(negative_samples) < num_neg_samples:
    d_neg = np.random.randint(0, num_drugs)
    m_neg = np.random.randint(0, num_microbes)
    n_neg = np.random.randint(0, num_diseases)

    if (d_neg, m_neg, n_neg) not in positive_set:  # Ensure it's a negative sample
        negative_samples.append([d_neg, m_neg, n_neg, 0])  # Label 0

# Merge positive and negative samples
negative_samples = np.array(negative_samples)
data = np.vstack((data, negative_samples))

# Shuffle dataset to avoid ordering bias
np.random.seed(42)
np.random.shuffle(data)

# Define train-test split (80% train, 20% test)
split_idx = int(0.8 * len(data))
train_indices, test_indices = data[:split_idx, :3], data[split_idx:, :3]
train_labels, test_labels = data[:split_idx, 3], data[split_idx:, 3]

# Function to transform input indices for the model
def transform(idxs):
    return [idxs[:, i] for i in range(idxs.shape[1])]

# Set random seeds for reproducibility
def set_session(seed=0):
    tf.random.set_seed(seed)
    np.random.seed(seed)

    gpus = tf.config.experimental.list_physical_devices('GPU')
    if gpus:
        try:
            for gpu in gpus:
                tf.config.experimental.set_memory_growth(gpu, True)
        except RuntimeError as e:
            print(e)

# Define CoSTCo Model with Dropout, L2 Regularization, and Noise Injection
def create_costco(shape, rank, nc):
    inputs = [k.Input(shape=(1,), dtype="int32") for i in range(len(shape))]
    embeds = [
        k.layers.Embedding(output_dim=rank, input_dim=shape[i])(inputs[i])
        for i in range(len(shape))
    ]
    x = k.layers.Concatenate(axis=1)(embeds)
    x = k.layers.Reshape(target_shape=(rank, len(shape), 1))(x)

    x = k.layers.Conv2D(
        nc // 2,
        kernel_size=(1, len(shape)),
        activation="relu",
        padding="valid"
    )(x)
    x = k.layers.Conv2D(
        nc // 2,
        kernel_size=(rank, 1),
        activation="relu",
        padding="valid"
    )(x)

    x = k.layers.Flatten()(x)
    x = k.layers.Dense(nc, activation="relu", kernel_regularizer=k.regularizers.l2(0.05))(x)
    x = k.layers.GaussianNoise(0.1)(x)
    x = k.layers.Dropout(0.5)(x)
    outputs = k.layers.Dense(1, activation="sigmoid")(x)

    model = k.Model(inputs=inputs, outputs=outputs)
    return model

# Training parameters
lr = 1e-4
rank = 10
nc = rank
epochs = 20
batch_size = 256
seed = 42
verbose = 1

# Set session
set_session(seed=seed)

# Model initialization and training
optim = k.optimizers.Adam(learning_rate=lr)
model = create_costco(shape, rank, nc)
model.compile(optimizer=optim, loss="binary_crossentropy", metrics=["accuracy"])

hists = model.fit(
    x=transform(train_indices),
    y=train_labels,
    verbose=verbose,
    epochs=epochs,
    batch_size=batch_size,
    validation_split=0.1,
    callbacks=[k.callbacks.EarlyStopping(
        monitor="val_loss",
        patience=5,
        restore_best_weights=True,
        mode='min')]
)

# Evaluate model
test_preds = model.predict(transform(test_indices), batch_size=batch_size).flatten()

# Convert predictions to binary format for classification metrics
test_preds_binary = (test_preds >= 0.5).astype(int)

# Compute Classification Metrics
def calculate_classification_metrics(y_true, y_pred, threshold=0.5):
    y_pred_binary = (y_pred >= threshold).astype(int)
    metrics = {
        "F1-score": f1_score(y_true, y_pred_binary),
        "Precision": precision_score(y_true, y_pred_binary),
        "Recall": recall_score(y_true, y_pred_binary),
        "ROC-AUC": roc_auc_score(y_true, y_pred) if len(np.unique(y_true)) > 1 else 0.0,
        "Average Precision": average_precision_score(y_true, y_pred)
    }
    return metrics

classification_metrics = calculate_classification_metrics(test_labels, test_preds)

# Compute Ranking Metrics
def hit_at_n(y_true, y_pred, N=5):
    sorted_indices = np.argsort(-y_pred)
    top_n = sorted_indices[:N]
    return 1 if any(y_true[i] for i in top_n) else 0

def ndcg_at_n(y_true, y_pred, N=5):
    sorted_indices = np.argsort(-y_pred)
    ideal_sorted_indices = np.argsort(-y_true)
    dcg = sum((y_true[i] / np.log2(idx + 2)) for idx, i in enumerate(sorted_indices[:N]))
    idcg = sum((y_true[i] / np.log2(idx + 2)) for idx, i in enumerate(ideal_sorted_indices[:N]))
    return dcg / idcg if idcg > 0 else 0

# Print Evaluation Metrics
for i in range(1,6,2):
    hit_n = hit_at_n(test_labels, test_preds, N=i)
    ndcg_n = ndcg_at_n(test_labels, test_preds, N=i)
    print(f"Hit@{i}: {hit_n:.4f}")
    print(f"NDCG@{i}: {ndcg_n:.4f}")

print("Classification Metrics:")
for metric, value in classification_metrics.items():
    print(f"{metric}: {value:.4f}")


Epoch 1/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 16s 347ms/step - accuracy: 0.5336 - loss: 1.0194 - val_accuracy: 0.5317 - val_loss: 1.0164
Epoch 2/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.4980 - loss: 1.0219 - val_accuracy: 0.5814 - val_loss: 1.0136
Epoch 3/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.5088 - loss: 1.0202 - val_accuracy: 0.6086 - val_loss: 1.0108
Epoch 4/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.5148 - loss: 1.0147 - val_accuracy: 0.6425 - val_loss: 1.0078
Epoch 5/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.4932 - loss: 1.0128 - val_accuracy: 0.6719 - val_loss: 1.0046
Epoch 6/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.5210 - loss: 1.0078 - val_accuracy: 0.7014 - val_loss: 1.0014
Epoch 7/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.4971 - loss: 1.0072 - val_accuracy: 0.7081 - val_loss: 0.9982
Epoch 8/20
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.5339 - loss: 0.9987 - val_accuracy: 0.7421 - va

### NeurTN Code

In [2]:
pip install torch_geometric

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.1/63.1 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 36.4 MB/s eta 0:00:00


In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
from torch_geometric.nn import GCNConv, global_mean_pool
from sklearn.metrics import f1_score, precision_score, recall_score, roc_auc_score, average_precision_score
from sklearn.model_selection import train_test_split

# Load dataset (Update path if necessary)
data_file = "/content/adj_del_4mic_myid.txt"
data = np.loadtxt(data_file).astype(int)

# Extract triplet indices and labels
d_indices = data[:, 0]  # Drug indices
m_indices = data[:, 1]  # Microbe indices
n_indices = data[:, 2]  # Disease indices
labels = data[:, 3]     # Association labels (0 or 1)

# Get dataset shape (number of unique drugs, microbes, diseases)
num_drugs = np.max(d_indices) + 1
num_microbes = np.max(m_indices) + 1
num_diseases = np.max(n_indices) + 1

# Convert to tensor format
triplets = torch.tensor(np.column_stack((d_indices, m_indices, n_indices)), dtype=torch.long)
labels = torch.tensor(labels, dtype=torch.float)

def generate_negative_samples(triplets, labels, num_samples):
    """
    Generate negative samples by randomly replacing microbes or diseases.
    """
    negative_triplets = triplets.clone().detach()
    for i in range(num_samples):
        idx = torch.randint(0, len(triplets), (1,))
        if torch.rand(1).item() < 0.5:
            negative_triplets[idx, 1] = torch.randint(0, num_microbes, (1,))  # Random microbe
        else:
            negative_triplets[idx, 2] = torch.randint(0, num_diseases, (1,))  # Random disease

    negative_labels = torch.zeros(num_samples, dtype=torch.float)
    return negative_triplets, negative_labels

# Generate an equal number of negative samples
neg_triplets, neg_labels = generate_negative_samples(triplets, labels, len(triplets))

# Combine positive and negative samples
triplets = torch.cat((triplets, neg_triplets), dim=0)
labels = torch.cat((labels, neg_labels), dim=0)


# Split data into train and test sets
train_triplets, test_triplets, train_labels, test_labels = train_test_split(triplets, labels, test_size=0.2, random_state=42)

# Move data to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
train_triplets, train_labels = train_triplets.to(device), train_labels.to(device)
test_triplets, test_labels = test_triplets.to(device), test_labels.to(device)


class NeurTN(nn.Module):
    def __init__(self, num_drugs, num_microbes, num_diseases, embed_dim=256, gnn_dim=128, mlp_dim=256):
        super(NeurTN, self).__init__()

        # Embedding layers
        self.drug_embedding = nn.Embedding(num_drugs, embed_dim)
        self.microbe_embedding = nn.Embedding(num_microbes, embed_dim)
        self.disease_embedding = nn.Embedding(num_diseases, embed_dim)

        # GNN for drug molecules
        self.gnn_conv1 = GCNConv(embed_dim, gnn_dim)
        self.gnn_conv2 = GCNConv(gnn_dim, gnn_dim)

        # MLP component
        self.mlp = nn.Sequential(
            nn.Linear(embed_dim * 3, mlp_dim),
            nn.ReLU(),
            nn.BatchNorm1d(mlp_dim),
            nn.Dropout(0.3),
            nn.Linear(mlp_dim, mlp_dim // 2),
            nn.ReLU(),
            nn.BatchNorm1d(mlp_dim // 2),
            nn.Dropout(0.2),
            nn.Linear(mlp_dim // 2, 1)
        )

    def forward(self, drug_idx, microbe_idx, disease_idx):
        # Embedding lookup
        drug_emb = self.drug_embedding(drug_idx)
        microbe_emb = self.microbe_embedding(microbe_idx)
        disease_emb = self.disease_embedding(disease_idx)

        # Feature fusion
        combined_emb = torch.cat([drug_emb, microbe_emb, disease_emb], dim=1)
        mlp_output = self.mlp(combined_emb)

        return torch.sigmoid(mlp_output)  # Sigmoid activation for binary classification


# Initialize model
model = NeurTN(num_drugs, num_microbes, num_diseases).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3, weight_decay=1e-5)
criterion = nn.BCELoss()  # Binary Cross Entropy Loss

# Training loop
epochs = 100
batch_size = 256

for epoch in range(epochs):
    model.train()
    optimizer.zero_grad()

    # Sample a mini-batch
    indices = torch.randperm(len(train_triplets))[:batch_size]
    batch_triplets = train_triplets[indices]
    batch_labels = train_labels[indices]

    predictions = model(batch_triplets[:, 0], batch_triplets[:, 1], batch_triplets[:, 2]).squeeze()
    loss = criterion(predictions, batch_labels)

    loss.backward()
    optimizer.step()

    print(f"Epoch {epoch+1}, Loss: {loss.item():.4f}")



# Evaluate model
model.eval()
with torch.no_grad():
    test_preds = model(test_triplets[:, 0], test_triplets[:, 1], test_triplets[:, 2]).cpu().numpy().flatten()
    test_preds_binary = (test_preds >= 0.5).astype(int)

# Compute Classification Metrics
def calculate_classification_metrics(y_true, y_pred):
    y_pred_binary = (y_pred >= 0.5).astype(int)
    metrics = {
        "F1-score": f1_score(y_true, y_pred_binary),
        "Precision": precision_score(y_true, y_pred_binary),
        "Recall": recall_score(y_true, y_pred_binary),
        "ROC-AUC": roc_auc_score(y_true, y_pred),
        "Average Precision": average_precision_score(y_true, y_pred)
    }
    return metrics

classification_metrics = calculate_classification_metrics(test_labels.cpu().numpy(), test_preds)

# Print Evaluation Metrics
print("\nClassification Metrics:")
for metric, value in classification_metrics.items():
    print(f"{metric}: {value:.4f}")


Epoch 1, Loss: 0.7281
Epoch 2, Loss: 0.7014
Epoch 3, Loss: 0.6931
Epoch 4, Loss: 0.6299
Epoch 5, Loss: 0.6290
Epoch 6, Loss: 0.6693
Epoch 7, Loss: 0.6267
Epoch 8, Loss: 0.6504
Epoch 9, Loss: 0.6054
Epoch 10, Loss: 0.6103
Epoch 11, Loss: 0.6355
Epoch 12, Loss: 0.6219
Epoch 13, Loss: 0.6339
Epoch 14, Loss: 0.5906
Epoch 15, Loss: 0.5805
Epoch 16, Loss: 0.5984
Epoch 17, Loss: 0.6241
Epoch 18, Loss: 0.5465
Epoch 19, Loss: 0.5707
Epoch 20, Loss: 0.6405
Epoch 21, Loss: 0.5638
Epoch 22, Loss: 0.5815
Epoch 23, Loss: 0.5694
Epoch 24, Loss: 0.6038
Epoch 25, Loss: 0.5869
Epoch 26, Loss: 0.5635
Epoch 27, Loss: 0.6010
Epoch 28, Loss: 0.5590
Epoch 29, Loss: 0.5644
Epoch 30, Loss: 0.5719
Epoch 31, Loss: 0.5259
Epoch 32, Loss: 0.5620
Epoch 33, Loss: 0.5641
Epoch 34, Loss: 0.5450
Epoch 35, Loss: 0.5357
Epoch 36, Loss: 0.5504
Epoch 37, Loss: 0.5150
Epoch 38, Loss: 0.5066
Epoch 39, Loss: 0.5671
Epoch 40, Loss: 0.5276
Epoch 41, Loss: 0.5312
Epoch 42, Loss: 0.5192
Epoch 43, Loss: 0.4820
Epoch 44, Loss: 0.49

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
pip install torch_geometric

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.1/63.1 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 33.3 MB/s eta 0:00:00


In [4]:
pip install torch-scatter

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.0/108.0 kB 3.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for torch-scatter: filename=torch_scatter-2.1.2-cp311-cp311-linux_x86_64.whl size=3622732 sha256=675de607588c6cf78ed82b2f51ec137246d1e906fe70f36f55812596b9d44156
  Stored in directory: /root/.cache/pip/wheels/b8/d4/0e/a80af2465354ea7355a2c153b11af2da739cfcf08b6c0b28e2
Successfully built torch-scatter


In [5]:
pip install xlrd

In [8]:
pip install rdkit

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.3/34.3 MB 31.9 MB/s eta 0:00:00


In [1]:
cd '/content/drive/MyDrive/MIDCN'

/content/drive/MyDrive/MIDCN


In [2]:
run main.py

/content/drive/MyDrive/MIDCN/process_smiles.py:38: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /pytorch/torch/csrc/utils/tensor_new.cpp:254.)
  GCNData = DATA.Data(x=torch.FloatTensor(features),
/usr/local/lib/python3.11/dist-packages/torch/optim/lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cuda:0 and cpu!